In [1]:
import numpy as np

In [2]:
### 走過所有路徑 並找出安全序列
def find_safe_arr(finsh_arr, all_allocation, all_need, available, history=None):
    history = [{
        'pids':                [],
        'origin_availables':   [],
        'release_availables':  []
    }] if history is None else history
    

    checked_pid = np.where((all_need <= available).all(axis=1))[0] # 這會找出小於available的pid
    checked_pid = np.setxor1d(np.where(finsh_arr==1), checked_pid) # 用finsh_arr去除掉已經走過的路徑
#     print(checked_pid)
    
    for pid in checked_pid:

        ### 走沒走過的地方
        fa = np.array(finsh_arr) # 把走過的路複製一份
        fa[pid] = 1 # 把現在的路標記

        ### 若為某一路徑分支  將路徑補齊
        if len(history[-1]['pids']) != len(np.where(finsh_arr==1)[0]):
            history[-1]['pids'].extend(history[-2]['pids'][:len(np.where(finsh_arr==1)[0])])
            history[-1]['origin_availables'].extend(history[-2]['origin_availables'][:len(np.where(finsh_arr==1)[0])])
            history[-1]['release_availables'].extend(history[-2]['release_availables'][:len(np.where(finsh_arr==1)[0])])

        history[-1]['pids'].append(pid)
        history[-1]['origin_availables'].append(list(available))
        history[-1]['release_availables'].append(list(available + all_allocation[pid]))

        find_safe_arr(fa, all_allocation, all_need, available + all_allocation[pid], history)

    
    if (finsh_arr == True).all():
        ### 如果為完整安全序列
        history.append({
            'pids':                [],
            'origin_availables':   [],
            'release_availables':  []
        })
    return history


### 發出請求
def a_request(pid, request, all_allocation, all_need, available):
    pid -= 1
    if (request <= all_need[pid]).all() and (request <= available).all():
        new_available            = np.copy(available) - request
        new_all_allocation       = np.copy(all_allocation)
        new_all_need             = np.copy(all_need)

        new_all_allocation[pid] += request
        new_all_need[pid]       -= request
        return True, new_all_allocation, new_all_need, new_available
    else:
        return False, all_allocation, all_need, available

## 依題目設定

In [3]:
all_allocation = np.array([
    [3, 1, 1, 1],
    [1, 0, 1, 0],
    [2, 2, 0, 0]
])

all_max = np.array([
    [3, 2, 1, 4],
    [2, 1, 2, 1],
    [4, 3, 2, 1]
])

In [4]:
all_need   = all_max - all_allocation
available  = np.array([12, 8, 5, 6])

## 1. 請列出 Allocation、Max、Available 及 Need 各執行緒序列之內容。

In [5]:
print('Allocation:\n', all_allocation)
print('Max:\n', all_max)
print('Available:\n', available)
print('Need:\n', all_need)

Allocation:
 [[3 1 1 1]
 [1 0 1 0]
 [2 2 0 0]]
Max:
 [[3 2 1 4]
 [2 1 2 1]
 [4 3 2 1]]
Available:
 [12  8  5  6]
Need:
 [[0 1 0 3]
 [1 1 1 1]
 [2 1 2 1]]


## 2. 請列出至少兩組的安全序列，例如：P0、P1、P2、P3、P4。

In [6]:
### 尋找安全序列
finsh      = np.zeros(all_need.shape[0])
history    = find_safe_arr(finsh, all_allocation, all_need, available)[:-1]
### 印出安全序列及經過的available
for his in history:
    print()
    print('執行程序(pid):', his['pids'])
    print('原始資源(available):', his['origin_availables'])
    print('資源釋放(available):', his['release_availables'])


執行程序(pid): [0, 1, 2]
原始資源(available): [[12, 8, 5, 6], [15, 9, 6, 7], [16, 9, 7, 7]]
資源釋放(available): [[15, 9, 6, 7], [16, 9, 7, 7], [18, 11, 7, 7]]

執行程序(pid): [0, 2, 1]
原始資源(available): [[12, 8, 5, 6], [15, 9, 6, 7], [17, 11, 6, 7]]
資源釋放(available): [[15, 9, 6, 7], [17, 11, 6, 7], [18, 11, 7, 7]]

執行程序(pid): [1, 0, 2]
原始資源(available): [[12, 8, 5, 6], [13, 8, 6, 6], [16, 9, 7, 7]]
資源釋放(available): [[13, 8, 6, 6], [16, 9, 7, 7], [18, 11, 7, 7]]

執行程序(pid): [1, 2, 0]
原始資源(available): [[12, 8, 5, 6], [13, 8, 6, 6], [15, 10, 6, 6]]
資源釋放(available): [[13, 8, 6, 6], [15, 10, 6, 6], [18, 11, 7, 7]]

執行程序(pid): [2, 0, 1]
原始資源(available): [[12, 8, 5, 6], [14, 10, 5, 6], [17, 11, 6, 7]]
資源釋放(available): [[14, 10, 5, 6], [17, 11, 6, 7], [18, 11, 7, 7]]

執行程序(pid): [2, 1, 0]
原始資源(available): [[12, 8, 5, 6], [14, 10, 5, 6], [15, 10, 6, 6]]
資源釋放(available): [[14, 10, 5, 6], [15, 10, 6, 6], [18, 11, 7, 7]]


## 3. 在某個時候，P3 請求系統給予 3 個 A 資源、2 個 B 資源及 1 個 C 資源。可否能被允許？

In [7]:
pid = 3
request = np.array([3, 2, 1, 0])

In [8]:
### 發出請求
chker, test_all_allocation, test_all_need, test_available = a_request(pid, request, all_allocation, all_need, available)
if chker:
    ### 若在合理範圍(即request小於need和available)

    ### 尋找安全序列(利用暫時修改變數進行安全序列檢查)
    finsh      = np.zeros(all_need.shape[0])
    history    = find_safe_arr(finsh, test_all_allocation, test_all_need, test_available)[:-1]
    
    if len(history) > 0:
        print('成功: request存在安全序列')
        ### 印出安全序列及經過的available
        for his in history:
            print()
            print('執行程序(pid):', his['pids'])
            print('原始資源(available):', his['origin_availables'])
            print('資源釋放(available):', his['release_availables'])

    else:
        print('失敗: request不存在安全序列')
else:
    print('失敗: request大於need或available')


失敗: request大於need或available
